In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# Build end-to-end Vespa apps with pyvespa

> Python API to create, modify, deploy and interact with Vespa applications

Our goal is to create, modify and deploy simple application packages using our python API. This enables us to run data analysis experiments that are fully integrated with Vespa. As an example, we want to create the application package we used in our [text search tutorial](https://docs.vespa.ai/documentation/tutorials/text-search.html). 

## Schema API

In [ ]:
from vespa.package import Document, Field, Schema, FieldSet, RankProfile, ApplicationPackage

document = Document(
    fields=[
        Field(name = "id", type = "string", indexing = ["attribute", "summary"]),
        Field(name = "title", type = "string", indexing = ["index", "summary"], index = "enable-bm25"),
        Field(name = "body", type = "string", indexing = ["index", "summary"], index = "enable-bm25")        
    ]
)

msmarco_schema = Schema(
    name = "msmarco", 
    document = document, 
    fieldsets = [FieldSet(name = "default", fields = ["title", "body"])],
    rank_profiles = [RankProfile(name = "default", first_phase = "nativeRank(title, body)")]
)

app_package = ApplicationPackage(name = "msmarco", schema=msmarco_schema)

## Deploy it locally

In [ ]:
from vespa.package import VespaDocker

vespa_docker = VespaDocker(application_package=app_package)
deployment_msg, app = vespa_docker.deploy(disk_folder="/Users/tmartins/projects/vespa/vespa/python/vespa/notebooks/sample_application")

In [ ]:
deployment_msg

In [ ]:
app

## Feed data to the app 

To feed data we need to specify the `schema` that we are sending data to. Each data point needs to have a unique `data_id` associated with it, independent of having an id field or not. The `fields` should be a dict containing all the fields in the schema. 

In [ ]:
response = app.feed_data_point(
    schema = "msmarco", 
    data_id = 1, 
    fields = {
        "id": "1", 
        "title": "This is a text", 
        "body": "This is the body of the text"
    }
)

In [ ]:
response.json()

## Change the application package and redeploy

We can add a new rank profile and redeploy our application

In [ ]:
app_package.schema.add_rank_profile(
    RankProfile(name = "bm25", inherits = "default", first_phase = "bm25(title) + bm25(body)")
)